In [1]:
# Import Library
import pandas as pd
import re

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait

from selenium.common.exceptions import TimeoutException

In [6]:
# Set Selenium settings
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

In [7]:
# Scrape web for stats
url = f"https://www.ourlads.com/nfldepthcharts/depthcharts.aspx"

PATH = "/Applications/chromedriver"
driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
wait = WebDriverWait(driver, 20)
driver.get(url)

wait.until(EC.presence_of_element_located((By.XPATH, "//table[@id='ctl00_phContent_gvChart']")))
driver.execute_script("window.stop();")

result = pd.read_html(driver.find_element(By.XPATH, value="//table[@id='ctl00_phContent_gvChart']").get_attribute("outerHTML"))
result = result[0]
result

In [44]:
df = result.copy()
df = df[['Team', 'Pos', 'Player 1', 'Player 2','Player 3', 'Player 4', 'Player 5']]

In [45]:
# Transform columns into rows
df1 = df[['Team', 'Pos', 'Player 1']]
df1 = df1.rename(columns={'Player 1':'Player'})
df1['posRank'] = "1"

df2 = df[['Team', 'Pos', 'Player 2']]
df2 = df2.rename(columns={'Player 2':'Player'})
df2['posRank'] = "2"

df3 = df[['Team', 'Pos', 'Player 3']]
df3 = df3.rename(columns={'Player 3':'Player'})
df3['posRank'] = "3"

df4 = df[['Team', 'Pos', 'Player 4']]
df4 = df4.rename(columns={'Player 4':'Player'})
df4['posRank'] = "4"

df5 = df[['Team', 'Pos', 'Player 5']]
df5 = df5.rename(columns={'Player 5':'Player'})
df5['posRank'] = "5"

complete = pd.concat([df1, df2, df3, df4, df5], axis=0, ignore_index=True)


In [66]:
# Select only relevant positions
posList = ['LWR', 'RWR', 'SWR', 'TE', 'QB', 'RB', 'PK', 'PR', 'KR']
filtered = complete.loc[complete['Pos'].isin(posList)]
# Convert WR roles to "WR"
filtered['Pos'].replace(["LWR", "RWR", "SWR"], "WR", inplace=True)
filtered['posRank'] = filtered['Pos'] + filtered['posRank']
filtered = filtered.reset_index(drop=True)
filtered.dropna(inplace=True)

/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [67]:
names = filtered['Player'].str.split(" ", n=2, expand=True)
names.columns = ['a', 'b', 'c']
names['a'] = names['a'].str.replace(",", "")
filtered['Player'] = names['b'] + " " + names['a']
filtered

,Team,Pos,Player,posRank
0,ARZ,WR,A.J. GREEN,WR1
1,ARZ,WR,Marquise Brown,WR1
2,ARZ,WR,Rondale Moore,WR1
3,ARZ,TE,ZACH ERTZ,TE1
5,ARZ,QB,Kyler Murray,QB1
...,...,...,...,...
1650,TEN,RB,Julius Chestnut,RB5
1657,WAS,TE,Curtis Hodges,TE5
1658,WAS,TE,Eli Wolf,TE5
1660,WAS,RB,Jaret Patterson,RB5


,index,Team,Pos,Player,posRank
0,3,ARZ,WR,"GREEN, A.J. U/Cin",WR1
1,4,ARZ,WR,"Brown, Marquise T/Bal",WR1
2,5,ARZ,WR,"Moore, Rondale 21/2",WR1
3,11,ARZ,TE,"ERTZ, ZACH T/Phi",TE1
5,13,ARZ,QB,"Murray, Kyler 19/1",QB1
...,...,...,...,...,...
1650,6144,TEN,RB,"Chestnut, Julius CF22",RB5
1657,6178,WAS,TE,"Hodges, Curtis CF22",TE5
1658,6179,WAS,TE,"Wolf, Eli SF22",TE5
1660,6181,WAS,RB,"Patterson, Jaret CF21",RB5
